In [ ]:
import numpy as np
import pandas as pd
from langdetect import detect, detect_langs
from pandarallel import pandarallel
import logging
from tqdm.notebook import tqdm
tqdm.pandas()

In [ ]:
#dataset = pd.read_csv("data/data_dic/d_reviews.csv")
def try_detect_language(text):
    try:
        lang = detect(text)
        return lang
    except:
        return None


pandarallel.initialize(progress_bar=True)
#dataset["comment_language"] = dataset['comments'].parallel_apply(lambda x: try_detect_language(x))
#pd.to_pickle(dataset, "data/pickles/draft_reviews/dic_reviews.pkl")
#dataset.to_csv("data/pickles/draft_reviews/dic_reviews.csv")

#dataset = pd.read_csv("data/pickles/draft_reviews/dic_reviews.csv", usecols=lambda x: x != "Unnamed: 0")
dataset = pd.read_pickle("data/pickles/draft_reviews/dic_reviews.pkl")
languages_needed = dataset['comment_language'].unique().tolist()

In [ ]:
copy_dataset = dataset.copy()
small_dataset = copy_dataset.loc[(copy_dataset['comment_language']!="en") & (copy_dataset['comment_language']!=np.nan), :]

In [ ]:
import argostranslate.package
import argostranslate.translate


to_code = "en"

argostranslate.package.update_package_index()
available_packages = argostranslate.package.get_available_packages()

for lang in languages_needed:
    try:
        package_to_install = next(
            filter(
                lambda x: x.from_code == lang and x.to_code == to_code, available_packages
            )
        )
        argostranslate.package.install_from_path(package_to_install.download())
    except:
        continue

In [ ]:
translatedText = argostranslate.translate.translate("Ciao, sono Filippo", "it", to_code)
print(translatedText)

In [ ]:
def translate_from_lan_to_eng(row):
    if len(row["comments"]) < 5:
        return "Text too short"
    try:
        lang = row["comment_language"]
        if lang == "zh-cn":
            lang = "zh-CN"
        if lang == "zh-tw":
            lang = "zh-TW"
        if lang == "he":
            lang = "iw"
        return argostranslate.translate.translate(row["comments"], lang, "en")
    except Exception as e:
        logging.error(f"Error translating row: {row}, error: {e}")
        return "Translation not applied"

In [ ]:
small_dataset["comment_translation"] = small_dataset.progress_apply(lambda x: translate_from_lan_to_eng(x), axis=1)